In [4]:
import pandas as pd
from prophet import Prophet

In [10]:
df = pd.read_csv('../data/MPS_actions/use_force/UF_aggregated_22_23.csv', index_col=0)

In [11]:
df.head()

,IncidentDate,Borough,counts
0,2022-04-01,Barking and Dagenham,9
1,2022-04-01,Barnet,7
2,2022-04-01,Bexley,5
3,2022-04-01,Brent,12
4,2022-04-01,Bromley,19


TODO: Данные не обработаны. есть лишние = "аэропорты", могут быть пропущенные данные для каких-то районов.

In [12]:
df = df.rename(columns={'IncidentDate': 'ds', 'counts':'y'})
df.head()

,ds,Borough,y
0,2022-04-01,Barking and Dagenham,9
1,2022-04-01,Barnet,7
2,2022-04-01,Bexley,5
3,2022-04-01,Brent,12
4,2022-04-01,Bromley,19


In [13]:
final = pd.DataFrame()
grouped = df.groupby('Borough')
for g in grouped.groups:
    # будет создаваться новый объект с элементами из этой группы
    group = grouped.get_group(g)
    # со значениями параметров по умолчанию 
    m = Prophet()    
    m.fit(group)        
    future = m.make_future_dataframe(periods=31)
    forecast = m.predict(future)    
    # если надо добавить столбцы '_lower', '_upper' 
    forecast = forecast.rename(columns={'yhat': g, 'yhat_lower': g+'_lower', 'yhat_upper': g+'_upper'})
    final = pd.merge(final, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
    
# мы добавили столбцы '_lower', '_upper'
final = final[[g+ads for g in grouped.groups.keys() for ads in ['', '_lower', '_upper']]] 

15:20:35 - cmdstanpy - INFO - Chain [1] start processing
15:20:35 - cmdstanpy - INFO - Chain [1] done processing
15:20:36 - cmdstanpy - INFO - Chain [1] start processing
15:20:36 - cmdstanpy - INFO - Chain [1] done processing
15:20:36 - cmdstanpy - INFO - Chain [1] start processing
15:20:37 - cmdstanpy - INFO - Chain [1] done processing
15:20:37 - cmdstanpy - INFO - Chain [1] start processing
15:20:37 - cmdstanpy - INFO - Chain [1] done processing
15:20:38 - cmdstanpy - INFO - Chain [1] start processing
15:20:38 - cmdstanpy - INFO - Chain [1] done processing
15:20:38 - cmdstanpy - INFO - Chain [1] start processing
15:20:38 - cmdstanpy - INFO - Chain [1] done processing
15:20:39 - cmdstanpy - INFO - Chain [1] start processing
15:20:39 - cmdstanpy - INFO - Chain [1] done processing
15:20:39 - cmdstanpy - INFO - Chain [1] start processing
15:20:39 - cmdstanpy - INFO - Chain [1] done processing
15:20:40 - cmdstanpy - INFO - Chain [1] start processing
15:20:40 - cmdstanpy - INFO - Chain [1]

In [15]:
final.reset_index(inplace=True);

In [16]:
final

,ds,Barking and Dagenham,Barking and Dagenham_lower,Barking and Dagenham_upper,Barnet,Barnet_lower,Barnet_upper,Bexley,Bexley_lower,Bexley_upper,...,Sutton_upper,Tower Hamlets,Tower Hamlets_lower,Tower Hamlets_upper,Waltham Forest,Waltham Forest_lower,Waltham Forest_upper,Wandsworth,Wandsworth_lower,Wandsworth_upper
0,2022-04-01,9.128435,3.242151,15.297652,10.311220,4.714252,15.937235,7.954021,3.052303,13.311828,...,9.080823,18.240335,10.651843,26.509557,9.619633,4.061894,15.288426,9.151135,4.407138,14.453631
1,2022-04-02,8.802809,3.280779,14.718846,10.169674,4.740082,15.807465,7.616138,2.296525,12.870918,...,8.083441,14.428430,5.830968,22.097968,10.464402,5.081878,16.040995,8.520657,2.999124,14.200294
2,2022-04-03,7.029450,0.493383,13.243813,9.745509,4.222141,14.970010,6.441776,0.551362,11.748040,...,7.253838,12.956210,5.374038,20.775577,9.225276,4.038884,14.710045,7.588335,2.008633,12.886285
3,2022-04-04,7.781425,1.294693,13.739513,10.168947,4.727172,15.822469,6.480230,0.732654,11.410417,...,7.959979,14.828229,6.999529,22.767686,8.941745,4.193149,14.182870,7.424202,2.208293,13.317662
4,2022-04-05,8.861117,2.941426,14.862283,10.380484,4.620101,16.349840,8.058770,2.753275,13.698140,...,8.798139,17.879446,10.240525,25.939465,9.375892,4.197107,14.702930,8.732533,3.373624,14.399801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,2023-04-27,9.738369,3.635526,15.852859,8.366748,2.583145,14.630157,9.469247,3.651322,14.860969,...,8.599246,16.760549,9.318088,24.512749,6.406522,0.932844,11.843854,6.437529,1.135041,11.691747
392,2023-04-28,10.198776,4.045492,16.180160,7.418221,1.685692,13.258394,9.566438,4.230354,15.207929,...,8.930690,15.910676,7.682339,23.532021,6.003710,0.340864,11.765903,7.088458,2.003302,12.433589
393,2023-04-29,9.873012,3.541895,16.012147,7.276674,1.539640,13.337779,9.228621,4.081122,14.683332,...,7.760426,12.104807,4.597946,20.242670,6.826547,1.438135,12.349015,6.457988,0.782527,12.260810
394,2023-04-30,8.099514,1.788047,14.231634,6.852508,0.838502,12.635700,8.054325,2.809821,13.554389,...,7.228162,10.638622,3.066394,18.119165,5.565488,0.003247,10.859746,5.525675,0.237100,10.891864


In [18]:
pwd

'g:\\github_projects\\neighbourhood-safety-prediction\\notebooks'

In [19]:
import joblib
import os

In [21]:
filename = 'dummy_prophet_model_01'
filepath = os.path.join("..","models", filename +".joblib")
joblib.dump(final, filepath)

['..\\models\\dummy_prophet_model_01.joblib']